# This is a mostly non-technical demo notebook for a CNN model that predicts March Madness games
##### Below are imports, more than are in use, just extra tools in case.

In [3]:
from __future__ import print_function
import sys
import csv
import tensorflow as tf
import sklearn
import en_core_web_sm
import keras
from keras import layers
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dropout, Activation
from keras.optimizers import SGD
from IPython import display
from keras.preprocessing.sequence import pad_sequences
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'monospace'
from matplotlib.ticker import MaxNLocator
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os, eli5, shap
np.set_printoptions(threshold=sys.maxsize)
from eli5.sklearn import PermutationImportance
from pdpbox import pdp

In [4]:
#Create dataframes that will be used for performance comparison later
pastModels=[[2017,1,.43857],[2017,2,.44981],[2017,3,.45373],[2017,4,.46100],[2017,5,.46107],[2018,1,.53194],[2018,2,.53693],[2018,3,.54013],[2018,4,.54967],[2018,5,.54987],[2019,1,.41477],[2019,2,.42012],[2019,3,.42698],[2019,4,.42788],[2019,5,.43148]]
cols=['Year','Rank','LogLoss']
pastScores=pd.DataFrame(pastModels, columns=cols)
pastMadness=[[2017,65.7,82,93,83],[2018,57,81,131,99.2],[2019,74,124,162,99.99]]
cols=['Year','avg_score','higherSeed_score','myScore','percentile_myScore']
pastMadness=pd.DataFrame(pastMadness, columns=cols)

## Importing the data here and slicing it in different ways for training and testing model
##### We will only be showing testing model performance

In [5]:
tourney_training_data=pd.read_csv("tourney_game_master1.csv")
#Get Training data
tourney_training_data.dropna(inplace=True)
tourneyInputData=tourney_training_data[['A_KP_AdjEM', 'A_KP_SOS_Adj', 'ALuck', 'Asos', 'Ats_pct', 'Aie', 'Afour', 'Amov', 'APOM', 'ASAG', 'AMOR', 'Aelo', 'Awins_top25', 'Awins_top5', 'Awin_pct', 'AAdjustD', 'AAdjustO', 'B_KP_AdjEM', 'B_KP_SOS_Adj', 'BLuck', 'Bsos', 'Bts_pct', 'Bie', 'Bfour', 'Bmov', 'BPOM', 'BSAG', 'BMOR', 'Belo', 'Bwins_top25', 'Bwins_top5', 'Bwin_pct', 'BAdjustD', 'BAdjustO','KP_AdjEM_diff', 'KP_SOS_Adj_diff', 'Luck_diff', 'sos_diff', 'ts_pct_diff', 'ie_diff', 'four_diff', 'mov_diff', 'POM_diff', 'SAG_diff', 'MOR_diff',  'elo_diff', 'wins_top25_diff', 'wins_top5_diff', 'win_pct_diff','AdjustD_diff','AdjustO_diff','Season','ATeam','BTeam']]

#held while scaling done

tourneyOutputData=tourney_training_data[['Season','AWon']]
train_input=tourneyInputData[tourneyInputData['Season']<2017].reset_index(drop=True)
train_input=train_input[['A_KP_AdjEM', 'A_KP_SOS_Adj', 'ALuck', 'Asos', 'Ats_pct', 'Aie', 'Afour', 'Amov', 'APOM', 'ASAG', 'AMOR', 'Aelo', 'Awins_top25', 'Awins_top5', 'Awin_pct', 'AAdjustD', 'AAdjustO', 'B_KP_AdjEM', 'B_KP_SOS_Adj', 'BLuck', 'Bsos', 'Bts_pct', 'Bie', 'Bfour', 'Bmov', 'BPOM', 'BSAG', 'BMOR', 'Belo', 'Bwins_top25', 'Bwins_top5', 'Bwin_pct', 'BAdjustD', 'BAdjustO','KP_AdjEM_diff', 'KP_SOS_Adj_diff', 'Luck_diff', 'sos_diff', 'ts_pct_diff', 'ie_diff', 'four_diff', 'mov_diff', 'POM_diff', 'SAG_diff', 'MOR_diff',  'elo_diff', 'wins_top25_diff', 'wins_top5_diff', 'win_pct_diff','AdjustD_diff','AdjustO_diff']]
train_output=tourneyOutputData[tourneyOutputData['Season']<2017].reset_index(drop=True)
train_output=train_output[['AWon']]


#Get test data
testTemp =tourneyInputData[tourneyInputData['Season']>2016].reset_index(drop=True)
testTemp=testTemp[['Season','ATeam','BTeam']]  
test_input=tourneyInputData[tourneyInputData['Season']>2016].reset_index(drop=True)
test_output=tourneyOutputData[tourneyOutputData['Season']>2016].reset_index(drop=True)


## Here we have a sample of some rows that will be used as inputs to predict the outcome of some March Madness games
##### Does not show all inputs, these inputs are an array of end of season stats for each team and then a comparison differential of said stats
###### Below we also have they testing output data which has the actual outcomes we will be trying to predict

In [6]:
#Show sample of DataFrame
test_input.head(5)

,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,Aie,Afour,Amov,APOM,ASAG,...,MOR_diff,elo_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjustD_diff,AdjustO_diff,Season,ATeam,BTeam
0,23.31,7.13,0.046,16.198486,57.770923,62.607934,0.367951,10.476471,20,18,...,-135,360.858019,3,0,0.203782,-8.4,14.8,2017,arizona,north-dakota
1,23.31,7.13,0.046,16.198486,57.770923,62.607934,0.367951,10.476471,20,18,...,-5,216.462950,3,0,0.007353,0.1,-0.3,2017,arizona,saint-marys
2,16.25,8.37,0.086,8.179100,55.557028,56.305476,0.357436,6.174118,38,37,...,6,-29.447322,-1,0,0.079044,4.0,6.7,2017,arkansas,seton-hall
3,7.89,-4.52,0.063,13.497683,57.716089,61.772175,0.386777,9.066667,88,97,...,157,616.586233,-6,0,0.059140,9.7,-6.6,2017,new-mexico-st,baylor
4,13.45,5.49,0.079,8.305611,55.716894,55.959719,0.348941,5.568182,61,55,...,32,-41.124048,-4,0,-0.046921,7.8,-2.9,2017,usc,baylor


In [7]:
test_output.head(5)



,Season,AWon
0,2017,1
1,2017,1
2,2017,1
3,2017,0
4,2017,0


## This scales all the data for inputs
##### It uses all tournament data collected to calculate standard deviation and then scales the test data as a subset putting mean at 0

In [8]:
#scale data
scaler = StandardScaler()

scaler.fit(train_input) 
X_train = pd.DataFrame(scaler.transform(train_input), index=train_input.index, columns=train_input.columns)
X_test = pd.DataFrame(scaler.transform(test_input.iloc[:,0:51]), index=test_input.iloc[:,0:51].index, columns=test_input.iloc[:,0:51].columns)
Y_train=train_output
Y_test=test_output

X_test['Season']=testTemp['Season']
X_test['ATeam'] =testTemp['ATeam']
X_test['BTeam']=testTemp['BTeam']


In [9]:
X_test.head(5)

,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,Aie,Afour,Amov,APOM,ASAG,...,MOR_diff,elo_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjustD_diff,AdjustO_diff,Season,ATeam,BTeam
0,0.573221,0.228608,0.657582,0.792645,1.132519,0.724414,0.191107,0.654301,-0.417829,-0.447897,...,-1.868418,0.984805,0.895360,-0.036743,1.449231,-1.188818,1.780350,2017,arizona,north-dakota
1,0.573221,0.228608,0.657582,0.792645,1.132519,0.724414,0.191107,0.654301,-0.417829,-0.447897,...,-0.085779,0.580750,0.895360,-0.036743,0.071174,-0.009504,-0.037048,2017,arizona,saint-marys
2,-0.224446,0.463203,1.550518,-0.578770,0.132103,-0.653210,-0.562810,-0.420664,-0.068412,-0.077132,...,0.065059,-0.107371,-0.257404,-0.036743,0.574128,0.531593,0.805454,2017,arkansas,seton-hall
3,-1.168992,-1.975445,1.037080,0.330774,1.107740,0.541729,1.540877,0.302054,0.902189,1.093706,...,2.135662,1.700400,-1.698359,-0.036743,0.434488,1.322427,-0.795301,2017,new-mexico-st,baylor
4,-0.540801,-0.081662,1.394254,-0.557135,0.204344,-0.728788,-1.171907,-0.572060,0.378064,0.274120,...,0.421587,-0.140046,-1.121977,-0.036743,-0.309587,1.058816,-0.349978,2017,usc,baylor


## Loading Model Framework and Weights
##### This loads a file that has the saved framework and tuned model, it then loads the weights which will be used for predicting games

In [10]:
#Load Model and weights
json_file = open('model_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
optimizer = keras.optimizers.Adagrad(lr=.01,epsilon=0,decay=0)
model.compile(optimizer= optimizer ,loss='binary_crossentropy',metrics=['acc'])
model.load_weights('model_3.h5')

print("Loaded model weights from disk")

Loaded model weights from disk


## How we show performance?
##### The primary basis for measuring a model performance is measuring 'Log Loss' or basically a measure of accuracy that factors in confidence
##### Below we will show: Log Loss for random choice of winner/confidence, The Log Loss for the Top 5 models as part of the Kaggle NCAA competition for the 3 years we are testing
##### Link to Kaggle Leaderboard for 2019: https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard

In [11]:
def LogLoss(predictions, realizations):
    predictions_use = predictions.clip(0)
    realizations_use = realizations.clip(0)
    LogLoss = -np.mean( (realizations_use * np.log(predictions_use)) + 
                        (1 - realizations_use) * np.log(1 - predictions_use) )
    return LogLoss

In [12]:
#For reference this is the LogLoss of a model that predicts 50/50 shot of either team winning each time
#rather than favoring one team or another

bench_5050 = np.repeat(0.5, len(Y_test))
LogLoss(bench_5050, Y_test.AWon)

0.6931471805599452

In [13]:
#Log Loss for past Kaggle competition winners
#https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard
pastScores

,Year,Rank,LogLoss
0,2017,1,0.43857
1,2017,2,0.44981
2,2017,3,0.45373
3,2017,4,0.46100
4,2017,5,0.46107
5,2018,1,0.53194
6,2018,2,0.53693
7,2018,3,0.54013
8,2018,4,0.54967
9,2018,5,0.54987


## How does my model stack up?
##### Here is a readout of accuracy followed by Log Loss for the 3 years of Kaggle competition shown above
##### We then show a sample of outputs for the models outputted prediction/confidence

In [14]:
#LogLoss FN for accuracy weighted by confidence


#Accuracy on Test set
years=[2017,2018,2019]
for year in years:
    X_test_temp=X_test[X_test['Season']==year].reset_index(drop=True)
    Y_test_temp=Y_test[Y_test['Season']==year].reset_index(drop=True)
    Y_test_temp=Y_test_temp[['AWon']]
    
    scores = model.evaluate(X_test_temp.iloc[:,0:51], Y_test_temp, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    #Test LogLoss on holdout test set
    y_pred_keras = model.predict(X_test_temp.iloc[:,0:51]).ravel()
    y_pred_round=np.round(y_pred_keras,0)
    temp = np.array(y_pred_keras)[np.newaxis]
    np.set_printoptions(suppress=True)
    y_pred_tran=temp.T
    print("LL_Test ",year,": ",LogLoss(y_pred_tran, Y_test_temp))


y_pred_keras = model.predict(X_test.iloc[:,0:51]).ravel()
y_pred_round=np.round(y_pred_keras,0)
testPerf=X_test.iloc[:,0:51]
testPerf['y_pred_confidence']=y_pred_keras

testPerf['y_pred_test']=y_pred_round
testPerf['y_act']=Y_test.AWon
testPerf.sample(10)

acc: 86.57%
LL_Test  2017 :  AWon    0.342599
dtype: float64
acc: 82.09%
LL_Test  2018 :  AWon    0.399666
dtype: float64
acc: 91.04%
LL_Test  2019 :  AWon    0.218012
dtype: float64


,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,Aie,Afour,Amov,APOM,ASAG,...,MOR_diff,elo_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjustD_diff,AdjustO_diff,y_pred_confidence,y_pred_test,y_act
143,-0.195070,-0.176256,-0.123738,-0.256785,-0.508785,-0.032257,-1.005122,-0.297014,-0.087825,-0.135674,...,0.435300,-2.173802,-2.274741,-3.488011,-0.759179,0.850702,-0.951765,0.036130,0.0,0
142,-2.421984,-1.659499,1.059403,-1.898370,0.488016,-1.963043,-2.638610,-1.888242,3.212222,3.201214,...,3.314946,-1.234453,-2.562932,-3.488011,-2.343339,2.931844,-1.517446,0.064006,0.0,0
4,-0.540801,-0.081662,1.394254,-0.557135,0.204344,-0.728788,-1.171907,-0.572060,0.378064,0.274120,...,0.421587,-0.140046,-1.121977,-0.036743,-0.309587,1.058816,-0.349978,0.105554,0.0,0
185,1.332473,0.994824,-0.280001,1.126582,0.605635,1.687786,-0.591309,1.051780,-0.650774,-0.643037,...,-0.113204,-0.266793,0.030787,-0.036743,-0.057789,-0.314738,-0.049084,0.863725,1.0,1
184,1.332473,0.994824,-0.280001,1.126582,0.605635,1.687786,-0.591309,1.051780,-0.650774,-0.643037,...,-0.264043,-1.563376,1.471742,-0.036743,-0.658051,-1.591173,-0.133334,0.971144,1.0,1
190,-0.104683,-0.427878,-1.373848,-0.314172,0.941298,-0.682246,-1.355435,-0.452811,-0.165473,0.059466,...,0.284461,-1.276458,0.030787,1.113680,-0.461872,-0.273116,-0.217585,0.904224,1.0,0
150,-0.198460,0.864284,-1.016674,-1.318176,-0.784160,-1.418931,-0.952622,-1.346881,-0.126649,-0.057618,...,0.558713,-1.052148,0.895360,-0.036743,-2.192706,0.975570,-1.108230,0.027911,0.0,0
125,0.907653,0.586176,-0.235355,1.032597,2.426238,0.766838,0.189220,1.218155,-0.709010,-0.701579,...,-0.099492,0.351446,0.030787,-0.036743,0.694884,0.753582,1.130419,0.908271,1.0,0
163,1.139270,1.011851,-0.391618,0.874690,-0.717581,0.628167,-3.216198,0.675403,-0.689598,-0.623523,...,-1.210213,-1.566212,1.471742,-0.036743,0.535439,-2.479127,0.853597,0.972586,1.0,1
186,1.651088,-0.276527,-0.391618,3.273278,3.030377,2.874596,0.620240,3.583487,-0.767246,-0.760120,...,-0.085779,1.206044,-1.121977,1.113680,0.697229,1.017193,1.250777,0.365356,0.0,0


# So that's cool, but how do we do at ACTUALLY picking a bracket?
#### We start with the 2017 bracket and show each rounds matchups as my model picked them, followed by the calculated ESPN score

In [15]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from runTournament import runTournament

df=pd.read_csv("tourney_game_master1.csv")
#2017 results
dfList, scoreDF=runTournament(df,2017,'model_3','model_3')
for df in dfList:
    print('\n',df)

scoreDF


                  ATeam              BTeam  ATeamID  BTeamID
0       mount-st-marys          villanova     1291     1437
1            wisconsin      virginia-tech     1458     1439
2       unc-wilmington           virginia     1423     1438
3              florida  east-tennessee-st     1196     1190
4                  usc                smu     1425     1374
5        new-mexico-st             baylor     1308     1124
6       south-carolina          marquette     1376     1266
7                 duke               troy     1181     1407
8      south-dakota-st            gonzaga     1355     1211
9           vanderbilt       northwestern     1435     1321
10           princeton         notre-dame     1343     1323
11            bucknell      west-virginia     1137     1452
12              xavier           maryland     1462     1268
13  florida-gulf-coast         florida-st     1195     1199
14                 vcu        saint-marys     1433     1388
15             arizona       north-dak

,round,score
0,32,29
1,16,24
2,8,24
3,4,16
4,2,0
5,1,0
6,Total,93


# 2017 might not have been the best year for us, how about 2018?
#### Same process as before

In [16]:
#2018 results
df=pd.read_csv("tourney_game_master1.csv")
dfList, scoreDF=runTournament(df,2018,'model_3','model_3')
for df in dfList:
    print('\n',df)

scoreDF


                     ATeam             BTeam  ATeamID  BTeamID
0               villanova           radford     1437     1347
1           virginia-tech           alabama     1439     1104
2               murray-st     west-virginia     1293     1452
3                marshall        wichita-st     1267     1455
4          st-bonaventure           florida     1382     1196
5              texas-tech  stephen-f-austin     1403     1372
6                  butler          arkansas     1139     1116
7        cal-st-fullerton            purdue     1168     1345
8                  kansas              penn     1242     1335
9       north-carolina-st        seton-hall     1301     1371
10                clemson     new-mexico-st     1155     1308
11  college-of-charleston            auburn     1158     1120
12                    tcu          syracuse     1395     1393
13            michigan-st          bucknell     1277     1137
14               oklahoma      rhode-island     1328     1348
15    

,round,score
0,32,29
1,16,22
2,8,16
3,4,16
4,2,16
5,1,32
6,Total,131


# We did ok considering a 16 seed upset a 1 seed in the 1st round 2018. Finally, this last year:

In [17]:
#2019 results
df=pd.read_csv("tourney_game_master1.csv")
dfList, scoreDF=runTournament(df,2019,'model_3','model_3')
for df in dfList:
    print('\n',df)

scoreDF



                 ATeam                BTeam  ATeamID  BTeamID
0     north-dakota-st                 duke     1295     1181
1                 vcu                  ucf     1433     1416
2      mississippi-st              liberty     1280     1251
3       virginia-tech          saint-louis     1439     1387
4            maryland              belmont     1268     1125
5                yale                  lsu     1463     1261
6           minnesota           louisville     1278     1257
7             bradley          michigan-st     1133     1277
8             gonzaga  fairleigh-dickinson     1211     1192
9            syracuse               baylor     1393     1124
10          marquette            murray-st     1266     1293
11            vermont           florida-st     1436     1199
12         arizona-st              buffalo     1113     1138
13         texas-tech    northern-kentucky     1403     1297
14            florida               nevada     1196     1305
15           michigan 

,round,score
0,32,28
1,16,30
2,8,32
3,4,24
4,2,16
5,1,32
6,Total,162


# 2019 went pretty well, but how well?
#### Below is my models score compared to the average ESPN score and the score given that you just pick higher seeds
#### Also shown is a percentile score compared to all other users on ESPN

In [18]:
pastMadness

,Year,avg_score,higherSeed_score,myScore,percentile_myScore
0,2017,65.7,82,93,83.00
1,2018,57.0,81,131,99.20
2,2019,74.0,124,162,99.99


# Now could this make me a millionaire?
#### Well, if I worked for Berkshire in 2019.....almost.
#### See this article: https://www.si.com/extra-mustard/2019/03/19/warren-buffett-ncaa-tournament-bracket-pool-million-dollars-year-sweet-16

# Finally, are bigger companies with lots of staff doing better?
#### Not quite, as far as I could find the only company publishing any kind of results is Adobe
#### There model achieve performance that garnered the 98th percentile, not quite 99.99...
#### See this page: https://www.adobe.com/analytics/hack-the-bracket.html?red=a